In [ ]:
%load_ext autoreload
%autoreload 2

import os
import psutil
from pathlib import Path
import numpy as np
import h5py
from scipy.spatial.distance import cdist
import nibabel as nib
from nibabel.affines import apply_affine

from mfs_tools.library.distance_stuff import make_distance_matrix, compare_mats


save_to = Path("/mnt/cache/pfm_python/")
reference_cifti_path = (
    save_to /
    "sub-ME01_task-rest_concatenated_and_demeaned_32k_fsLR.dtseries.nii"
)
surface_files = {
    'lh': Path(
        "/mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k"
        "/ME01.L.midthickness.32k_fs_LR.surf.gii"
    ),
    'rh': Path(
        "/mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k"
        "/ME01.R.midthickness.32k_fs_LR.surf.gii"
    ),
}
wb_command_path = "/usr/local/workbench/2.0.1/bin_linux64/wb_command"
work_dir = None


In [ ]:
# We need to build a complete 85059 x 85059 distance matrix in parts:
# A complete fsLR can have up to 32492 vertices per hemisphere, but
# Lynch's actual data avoids filling the medial wall.

#                   cortex                 subcortex
#           left hem       right hem
#     🭽                                                🭾
#       🭽    (1)     🭾 🭽    (3)     🭾 🭽    (2)     🭾
#   lh   29696 x 29696   29696 x 29716   29696 x 25647
#       🭼            🭿 🭼            🭿 🭼            🭿
#       🭽    (3)     🭾 🭽    (1)     🭾 🭽    (2)     🭾
#   rh   29716 x 29696   29716 x 29716   29716 x 25647
#       🭼            🭿 🭼            🭿 🭼            🭿
#       🭽    (2)     🭾 🭽    (2)     🭾 🭽    (2)     🭾
#   sc   25647 x 29696   25647 x 29716   25647 x 25647
#       🭼            🭿 🭼            🭿 🭼            🭿
#     🭼                                                🭿

# If we built it all at once, it would have 7 billion 32-bit floats,
# requiring 28GB just to hold it. Debugging this code on a VM was consuming
# about 140GB RAM, but by building distance matrices in pieces, and
# converting each to uint8 as its built allowed me to run this on my
# workstation with peak memory usage of about 54GB.

----

## Calculate left and right cortical distances

----

In [ ]:
# (1) Start by either loading pre-built single-hemisphere distance matrices
#     or building them.

mem_before = psutil.Process(os.getpid()).memory_info().rss

distance_matrices = dict()
for (hemi) in ('lh', 'rh', ):
    mat_file = Path(save_to) / f"dist_{hemi}.npy"
    if mat_file.exists():
        distance_matrices[hemi] = np.load(mat_file)
    else:
        distance_matrices[hemi] = make_distance_matrix(
            reference_cifti_path,
            surface_files[hemi],
            save_to,
            num_procs=12,
            wb_command_path=wb_command_path,
            work_dir=Path(save_to) / "tmp",
        )
        print(f"built {distance_matrices[hemi].shape}-shaped distance matrix")
        np.save(mat_file, distance_matrices[hemi])

py_lh = distance_matrices["lh"]
py_rh = distance_matrices["rh"]

mem_after = psutil.Process(os.getpid()).memory_info().rss
print(f"Mem before {mem_before / 1024 / 1024:0,.1f}MB; "
      f"Mem after {mem_after / 1024 / 1024:0,.1f}MB; "
      f"delta {(mem_after - mem_before) / 1024 / 1024:0,.1f}")

Matlab
- running with 5 workers, finished in 27:53

Python
- Running with num_procs == 3 finished in 19:38.5
- Running with num_procs == 15 finished in 7:44.9
- Running with num_procs == 12 finished in 8:32 & 8:28; 7:44 & 7:42; 7:21 & 7:16
- Running with 5 workers, finished in 12:57 and 13:30


In [ ]:
# Load the matrices from matlab

# Ensure our lh and rh match the lh and rh from matlab.
# Then ensure our combined cortical d matches, too.

mem_before = psutil.Process(os.getpid()).memory_info().rss

matlab_outdir = Path("/mnt/cache/pfm_matlab/")
lh_matlab_file = h5py.File(matlab_outdir / "lh.mat", 'r')
rh_matlab_file = h5py.File(matlab_outdir / "rh.mat", 'r')
ml_lh = np.array(lh_matlab_file.get('lh'), dtype=np.uint8)
ml_rh = np.array(rh_matlab_file.get('rh'), dtype=np.uint8)
print(f"The lh.mat from matlab contains {ml_lh.shape} {str(ml_lh.dtype)}s.")
print(f"The rh.mat from matlab contains {ml_rh.shape} {str(ml_rh.dtype)}s.")

mem_after = psutil.Process(os.getpid()).memory_info().rss
print(f"Mem before {mem_before / 1024 / 1024:0,.1f}MB; "
      f"Mem after {mem_after / 1024 / 1024:0,.1f}MB; "
      f"delta {(mem_after - mem_before) / 1024 / 1024:0,.1f}")


----

## Calculate subcortical, and subcortex to left and right cortical distances

----

In [ ]:
# Get gifti coordinates to calculate Euclidean distance between them.

mem_before = psutil.Process(os.getpid()).memory_info().rss

# Get the anatomical brain axis from the reference Cifti2 image
ref_img = nib.cifti2.Cifti2Image.from_filename(reference_cifti_path)
brain_ax = ref_img.header.get_axis(1)
print(f"Length of cifti2 brain_axis: {len(brain_ax)}")

# Extract the 3D Cartesian coordinates of all surface vertices
lh_surface_img = nib.gifti.gifti.GiftiImage.from_filename(surface_files['lh'])
rh_surface_img = nib.gifti.gifti.GiftiImage.from_filename(surface_files['rh'])
print("Gifti Surface coordinates: "
      f"[{lh_surface_img.darrays[0].data.shape} + "
      f"{rh_surface_img.darrays[0].data.shape}]")

# Extract the vertex indices into the mapped BOLD data
anat_map = {
    'CortexLeft': 'CIFTI_STRUCTURE_CORTEX_LEFT',
    'CortexRight': 'CIFTI_STRUCTURE_CORTEX_RIGHT',
}
lh_surf_anat = lh_surface_img.darrays[0].metadata.get('AnatomicalStructurePrimary', '')
lh_surf_idx = brain_ax[brain_ax.name == anat_map[lh_surf_anat]]
lh_surf_coords = lh_surface_img.darrays[0].data[lh_surf_idx.vertex, :]
print(f"Just vertices in {str(type(lh_surf_idx))} {lh_surf_anat}: {len(lh_surf_idx)}")
rh_surf_anat = rh_surface_img.darrays[0].metadata.get('AnatomicalStructurePrimary', '')
rh_surf_idx = brain_ax[brain_ax.name == anat_map[rh_surf_anat]]
rh_surf_coords = rh_surface_img.darrays[0].data[rh_surf_idx.vertex, :]
print(f"Just vertices in {str(type(rh_surf_idx))} {rh_surf_anat}: {len(rh_surf_idx)}")

# Get the subcortical voxels, too, from a volumetric grid rather than vertices.
# Note that python's voxel locations are consistently shifted relative to
# matlab's. Python's x values are ml+2mm, y=ml+2mm, z=ml-2mm.
# Maybe 0-based vs 1-based indexing, then multiplied by the affine?
# Maybe it's start of voxel vs end of voxel, not center?
# It's all relative, so the subcortex-to-subcortex distances are identical,
# and distance differences are only between subcortical and cortical.
# I've added a 1 below to ensure these data match Lynch's for now.
# I also checked for left-right biases in another notebook to see
# whether the Lynch matlab distance matrix or the Schmidt python distance
# matrix have more left/right bias. Lynch's is less biased, so I think
# adding 1 is the correct approach.
ctx_labels = list(anat_map.values())
subcortical_coordinates = apply_affine(
    brain_ax.affine,
    brain_ax.voxel[~np.isin(brain_ax.name, ctx_labels)] + 1,
)
print("Nifti subcortical coordinates: "
      f" = {subcortical_coordinates.shape}")
print("Cifti cortical coordinates: "
      f" = {lh_surf_coords.shape} & {rh_surf_coords.shape}")

whole_brain_coordinates = np.vstack([
    lh_surf_coords, rh_surf_coords, subcortical_coordinates
])
print("Whole brain coordinates: "
      f" = {whole_brain_coordinates.shape}")


mem_after = psutil.Process(os.getpid()).memory_info().rss
print(f"Mem before {mem_before / 1024 / 1024:0,.1f}MB; "
      f"Mem after {mem_after / 1024 / 1024:0,.1f}MB; "
      f"delta {(mem_after - mem_before) / 1024 / 1024:0,.1f}")


In [ ]:
# (2) Now, calculate the Euclidean distances between subcortical voxels
#     and everywhere. Doing this in three chunks avoids duplication of
#     memory, but takes a little longer (seconds, not minutes).

mem_before = psutil.Process(os.getpid()).memory_info().rss

sc_to_lh_dist = np.uint8(
    np.clip(
        cdist(subcortical_coordinates, lh_surf_coords) + 0.5,
        0, 255
    )
)
sc_to_rh_dist = np.uint8(
    np.clip(
        cdist(subcortical_coordinates, rh_surf_coords) + 0.5,
        0, 255
    )
)
sc_to_sc_dist = np.uint8(
    np.clip(
        cdist(subcortical_coordinates, subcortical_coordinates) + 0.5,
        0, 255
    )
)
print(f"Euclidean distance matrices: {sc_to_lh_dist.shape}, {sc_to_rh_dist.shape}, {sc_to_sc_dist.shape}")

mem_after = psutil.Process(os.getpid()).memory_info().rss
print(f"Mem before {mem_before / 1024 / 1024:0,.1f}MB; "
      f"Mem after {mem_after / 1024 / 1024:0,.1f}MB; "
      f"delta {(mem_after - mem_before) / 1024 / 1024:0,.1f}")


In [ ]:
# This works! but it will generate a giant matrix, using about 146GB of RAM.
# I verified that all of the piecing together works on a large VM, so
# we can confidently do pieces at a time, then uint8 them and stitch them together.

# Running this code on a server with >146GB RAM
"""
sc_to_ctx_dist = cdist(subcortical_coordinates, used_surface_coordinates)
ctx_to_sc_dist = cdist(used_surface_coordinates, subcortical_coordinates)
sc_to_sc_dist = cdist(subcortical_coordinates, subcortical_coordinates)
euclid_dist = cdist(
    whole_brain_coordinates, whole_brain_coordinates,
)

tmp = euclid_dist[used_surface_coordinates.shape[0]:, :used_surface_coordinates.shape[0]]
print(f"Compare SC-to-CTX part {sc_to_ctx_dist.shape} to part of the whole {tmp.shape}")
print("  full " + "match" if np.allclose(sc_to_ctx_dist, tmp) else "miss")
print("  uint8 " + "match" if np.allclose(np.astype(sc_to_ctx_dist, np.uint8), np.astype(tmp, np.uint8)) else "miss")

tmp = euclid_dist[used_surface_coordinates.shape[0]:, used_surface_coordinates.shape[0]:]
print(f"Compare SC-to-SC part {sc_to_sc_dist.shape} to part of the whole {tmp.shape}")
print("  full " + "match" if np.allclose(sc_to_sc_dist, tmp) else "miss")
print("  uint8 " + "match" if np.allclose(np.astype(sc_to_sc_dist, np.uint8), np.astype(tmp, np.uint8)) else "miss")

tmp = euclid_dist[:used_surface_coordinates.shape[0], used_surface_coordinates.shape[0]:]
print(f"Compare CTX-to-SC part {ctx_to_sc_dist.shape} to part of the whole {tmp.shape}")
print("  full " + "match" if np.allclose(ctx_to_sc_dist, tmp) else "miss")
print("  uint8 " + "match" if np.allclose(np.astype(ctx_to_sc_dist, np.uint8), np.astype(tmp, np.uint8)) else "miss")

print(f"Compare CTX-to-SC part {ctx_to_sc_dist.shape} to transposed SC-to-CTX {sc_to_ctx_dist.T.shape}")
print("  full " + "match" if np.allclose(ctx_to_sc_dist, sc_to_ctx_dist.T) else "miss")
print("  uint8 " + "match" if np.allclose(np.astype(ctx_to_sc_dist, np.uint8), np.astype(sc_to_ctx_dist.T, np.uint8)) else "miss")
"""

# Results in
"""
Compare SC-to-CTX part (25647, 59412) to part of the whole (25647, 59412)
  full match
  uint8 match
Compare SC-to-SC part (25647, 25647) to part of the whole (25647, 25647)
  full match
  uint8 match
Compare CTX-to-SC part (59412, 25647) to part of the whole (59412, 25647)
  full match
  uint8 match
Compare CTX-to-SC part (59412, 25647) to transposed SC-to-CTX (59412, 25647)
  full match
  uint8 match
"""

----

## Put all cortical distances together

----

In [ ]:
# We've calculated most of the distance matrix piece-meal, and need to
# fill in some missing sections with fake large values that will exceed
# the distance threshold and cause removal of those connectivity values later.
# Start pasting lh-lh and rh-rh distances into a complete distance matrix
# where anything between lh and rh is "large".
# The largest uint8 is 2^8 == 256, which is big enough to get masked later.
# [ [ lh  ] [255s ] [lh-sc] ]
# [ [255s ] [ rh  ] [rh-sc] ]
# [ [sc-lh] [sc-rh] [ sc  ] ]

mem_before = psutil.Process(os.getpid()).memory_info().rss

# (3) Create two filler blocks
top_mid_lh_rh = np.ones((py_lh.shape[0], py_rh.shape[1]), dtype=np.uint8) * 255
mid_mid_rh_lh = np.ones((py_rh.shape[0], py_lh.shape[1]), dtype=np.uint8) * 255

# Put complete distance matrix together with real and filler data
py_complete = np.vstack([
    np.hstack([py_lh, top_mid_lh_rh, sc_to_lh_dist.T, ]),
    np.hstack([mid_mid_rh_lh, py_rh, sc_to_rh_dist.T, ]),
    np.hstack([sc_to_lh_dist, sc_to_rh_dist, sc_to_sc_dist, ])  # bottom row of blocks
])

np.save(Path(save_to) / "dist_complete.npy", py_complete)

mem_after = psutil.Process(os.getpid()).memory_info().rss
print(f"Mem before {mem_before / 1024 / 1024:0,.1f}MB; "
      f"Mem after {mem_after / 1024 / 1024:0,.1f}MB; "
      f"delta {(mem_after - mem_before) / 1024 / 1024:0,.1f}")


In [ ]:
# Load matlab-based distance matrix for comparison

matlab_dist_file = h5py.File(matlab_outdir / "DistanceMatrix.mat", 'r')
ml_complete = np.array(matlab_dist_file.get('D'), dtype=np.uint8)


In [ ]:
# Compare matlab's distance results to python's.
# We know there are very few, << 1%, float representation differences
# that were locked in rounding to uint8s. Everything else should match.
mat_sections = [("lh", 0, 29696), ("rh", 29696, 59412), ("sc", 59412, 85059), ]
for (a_name, a_start, a_end) in mat_sections:
    for (b_name, b_start, b_end) in mat_sections:
        section_name = "-".join([a_name, b_name])
        mat_a = py_complete[a_start:a_end, b_start:b_end]
        mat_b = ml_complete[a_start:a_end, b_start:b_end]
        print(f"** Compare {section_name}: {mat_a.shape} v {mat_b.shape}")
        compare_mats(mat_a, mat_b,
                     a_name=f"python {section_name}",
                     b_name=f"matlab {section_name}",
                     verbose=True, preview=False)
